code orionated from:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

my main point in modifying this code is to make it notebook runable. the idea is that often when somebody is new to Tesorflow it is difficult to isolate a protion of code that is causing the problem. It is my hope that this notebook will assist people to learn Tensorflow with less struglling with each error. It is possible that this might just run, but if it crashes it will allow you to more easily find the cause.

In [38]:
from platform import python_version
python_version()


'3.5.2'

In [39]:

from __future__ import print_function

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Set parameters

In [40]:
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

Network Parameters

In [41]:
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

tf Graph input
Place holders are used to pretend to be tensors. Laters data will be fed into them.

In [42]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

Create some wrappers for simplicity. The idea is to make the Tensorflow calls more of normal function calls.

In [43]:
def conv2d(x, W, b, strides=1):
    '''Conv2D wrapper, with bias and relu activation'''
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    '''MaxPool2D wrapper '''
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
padding='SAME')

Create model

In [44]:
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

Store layers weight & bias

In [45]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

Construct model

In [46]:
pred = conv_net(x, weights, biases, keep_prob)

Define loss and optimizer

In [47]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Evaluate model

In [48]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Initializing the variables

In [49]:
init = tf.global_variables_initializer()

In [52]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
keep_prob: 1.}))

Iter 1280, Minibatch Loss= 27259.310547, Training Accuracy= 0.27344
Iter 2560, Minibatch Loss= 9159.318359, Training Accuracy= 0.61719
Iter 3840, Minibatch Loss= 6457.607910, Training Accuracy= 0.70312
Iter 5120, Minibatch Loss= 3051.637451, Training Accuracy= 0.80469
Iter 6400, Minibatch Loss= 3381.936523, Training Accuracy= 0.83594
Iter 7680, Minibatch Loss= 3526.810791, Training Accuracy= 0.82812
Iter 8960, Minibatch Loss= 2951.521484, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 3817.645752, Training Accuracy= 0.83594
Iter 11520, Minibatch Loss= 2377.578857, Training Accuracy= 0.85156
Iter 12800, Minibatch Loss= 2481.777832, Training Accuracy= 0.85938
Iter 14080, Minibatch Loss= 1549.543823, Training Accuracy= 0.91406
Iter 15360, Minibatch Loss= 2102.523438, Training Accuracy= 0.91406
Iter 16640, Minibatch Loss= 764.491089, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 3682.156006, Training Accuracy= 0.81250
Iter 19200, Minibatch Loss= 2167.090332, Training Accur